<a href="https://colab.research.google.com/github/vijjus/CV/blob/master/IAMwords_CRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
load_weights = False
do_data_analysis = False
IMG_SIZE=(32, 128)
weigths_file = "/content/gdrive/MyDrive/EffNetRNN_IAMWords.pth"

In [ ]:
!nvidia-smi

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
!pip install colorama

In [ ]:
import os
import cv2
import csv
import sys
import torch
import collections
import editdistance
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from colorama import Fore
from torch import nn, einsum
import torch.nn.functional as F
from itertools import groupby
import albumentations as A
from google.colab import drive
import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet
from albumentations.pytorch import ToTensor
from random import shuffle
import matplotlib.pyplot as plt
from typing import Callable, Dict, List, Set
from torch.nn import Conv2d, Dropout, LogSoftmax
from torch.utils.data.dataloader import default_collate

In [ ]:
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
!cp /content/gdrive/MyDrive/words.tgz .

In [ ]:
!mkdir data; mv words.tgz data/ ; cd data ; tar -zxf words.tgz; ls ; cd -

In [ ]:
!rm ./data/a01/a01-117/a01-117-05-02.png
!rm ./data/r06/r06-022/r06-022-03-05.png

In [ ]:
img = cv2.imread('data/a01/a01-000u/a01-000u-05-00.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray')

In [ ]:
!cp /content/gdrive/MyDrive/ascii.tgz .

In [ ]:
!tar -zxf ascii.tgz

In [ ]:
!ls

In [ ]:
!tail -10 words.txt

In [ ]:
def list_files_and_basename_recursive(dirpath, ext='.png'):
    # TODO unit test
    retlist = {}
    for root, dirs, files in os.walk(dirpath):
        for filename in files:
            if not filename.endswith(ext):
                continue

            image_filename_no_ext = os.path.basename(os.path.splitext(filename)[0])
            retlist[image_filename_no_ext] = os.path.join(root, filename)

    return retlist


In [ ]:
all_files = list_files_and_basename_recursive('./data')

In [ ]:
all_keys = list(all_files.keys())
num_total = len(all_keys)
print(f'There are {num_total} words in all..')
shuffle(all_keys)
num_train = int(0.8 * num_total)
train_words = all_keys[:num_train]
test_words = all_keys[num_train:]

In [ ]:
WORD_LEVEL_FIELDNAMES = [
        "word_id",
        "word_segmentation_result",
        "graylevel",
        "bbox_x",
        "bbox_y",
        "bbox_w",
        "bbox_h",
        "tag",
        "label",
    ]
all_labels = dict()
with open('words.txt', 'r') as fp:
  # QUOTE_NONE is required here, there are some open double quotes in the file that screw with parsing
  reader = csv.DictReader(
      fp,
      fieldnames=WORD_LEVEL_FIELDNAMES,
      delimiter=" ",
      quoting=csv.QUOTE_NONE,
  )
  # skip the first 18 lines
  for i in range(18):
    next(reader)
  for row in reader:
    all_labels[row['word_id']] = row['label']

In [ ]:
def write_meta(file_name, word_list):
  with open(file_name, 'w') as fp:
    count = 0
    for entry in word_list:
      fp.write(f'{all_labels[entry]} {all_files[entry]}'+'\n')
      count += 1
    print(f'Wrote {count} entries to {file_name}')

In [ ]:
write_meta('train.txt', train_words)

In [ ]:
write_meta('test.txt', test_words)

In [ ]:
!tail -10 train.txt

In [ ]:
class Vocabulary(object):
    """Convert between str and label.
    NOTE:
        Insert `blank` to the alphabet for CTC.
    Args:
        alphabet (Iterable): set of the possible characters.
        ignore_case (bool, default=False): whether or not to ignore all of the case.
    """

    def __init__(self, alphabet: List):
        self.alphabet = alphabet

        self.dict = {}
        for i, char in enumerate(alphabet):
            self.dict[char] = i

    @property
    def num_classes(self):
        return len(self.dict)

    def encode(self, text):
        """Support batch or single str.
        Args:
            text (str or list of str): texts to convert.
        Returns:
            torch.IntTensor [length_0 + length_1 + ... length_{n - 1}]: encoded texts.
            torch.IntTensor [n]: length of each text.
        """
        if isinstance(text, str):
            text = [self.dict[char] for char in text]
            length = [len(text)]
        elif isinstance(text, torch.Tensor):
            if text.dim() == 2:
                label = []
                length = []
                for sub in text:
                    label_char = [self.dict[str(x)] for x in sub.tolist()]
                    label.append(label_char)
                    length.append(len(sub))
                text = label
            else:
                text = [self.dict[str(x)] for x in text.tolist()]
                length = [len(text)]
        elif isinstance(text, collections.Iterable):
            length = [len(s) for s in text]
            text = "".join(text)
            text, _ = self.encode(text)
        else:
            raise ValueError(f"Invalid input type {type(text)}")

        return torch.IntTensor(text), torch.IntTensor(length)

    def decode(self, t, length, raw=False):
        """Decode encoded texts back into strs.
        Args:
            torch.IntTensor [length_0 + length_1 + ... length_{n - 1}]: encoded texts.
            torch.IntTensor [n]: length of each text.
        Raises:
            AssertionError: when the texts and its length does not match.
        Returns:
            text (str or list of str): texts to convert.
        """
        if length.numel() == 1:
            length = length[0]
            assert (
                t.numel() == length
            ), "text with length: {} does not match declared length: {}".format(
                t.numel(), length
            )
            if raw:
                return "".join(
                    [self.alphabet[i] if self.alphabet[i] != "<SEP>" else "" for i in t]
                )
            else:
                char_list = []
                for i in range(length):
                    if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])):
                        c = self.alphabet[t[i]]
                        if c == "<SEP>":
                            c = " "
                        char_list.append(c)
                return "".join(char_list)
        else:
            # batch mode
            assert (
                t.numel() == length.sum()
            ), "texts with length: {} does not match declared length: {}".format(
                t.numel(), length.sum()
            )
            texts = []
            index = 0
            for i in range(length.numel()):
                label_length = length[i]
                texts.append(
                    self.decode(
                        t[index : index + label_length],
                        torch.IntTensor([label_length]),
                        raw=raw,
                    )
                )
                index += label_length
            return texts

In [ ]:
def base_collator_with_padding(
    pad_key, base_collator_fn=default_collate, pad_val=0, pad_idx=0
):
    """Pad: a given key in the data iterator output to a consistent length.
    Otherwise, falls back to default collate functionality
    Args:
        pad_key: str The specific data stream to pad
        base_collator_fn: Callable The default collator to call. Defaults to default_collate from PyTorch
        pad_val: Any The value to pad in the tensor. Defaults to 0
    Returns:
        The result of collation, including padding of the variable length streams
    """

    def fn(batch):
        max_len = max(item[pad_key].shape[pad_idx] for item in batch)

        for item in batch:
            original_tensor = item[pad_key]

            # TODO currently assumes a single dimension tensor
            padded_tensor = torch.full((max_len,), pad_val, dtype=original_tensor.dtype)
            padded_tensor[: original_tensor.shape[pad_idx]] = original_tensor

            item[pad_key] = padded_tensor

        return base_collator_fn(batch)

    return fn

In [ ]:
def resize_image(image, img_size=IMG_SIZE):
    if isinstance(image, str):
      image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    scale_coef = float(img_size[0]) / image.shape[0]
    new_width = int(image.shape[1] * scale_coef)

    if new_width > img_size[1]:
        # TODO this is a hack
        resized_image = cv2.resize(
            image, (img_size[1], img_size[0]), cv2.INTER_AREA
        )

        resized_image = resized_image
        return np.reshape(resized_image, img_size)

    new_height = img_size[0]
    resized_image = cv2.resize(image, (new_width, new_height), cv2.INTER_CUBIC)

    padded_image = np.full(img_size, 255, dtype=image.dtype)

    padded_image[
        : resized_image.shape[0], : resized_image.shape[1]
    ] = resized_image

    padded_image = padded_image
    padded_image = np.reshape(padded_image, img_size)
    return padded_image

In [ ]:
class IAMDataset(torch.utils.data.Dataset):

    def __init__(
        self,
        words_file: str,
        transform: Callable = None,
        max_sequence_len=None,
    ):
        self._data = []
        self._transform = transform

        chars: Set[str] = set()

        self._limit_max_sequence_length = bool(max_sequence_len)

        # either set it to our upper limit, or initialize it to be overridden by our longest sequence
        self._max_sequence_length = (
            max_sequence_len if self._limit_max_sequence_length else -1
        )

        with open(words_file) as fp:
            for line in fp:
                entries = line.split()
                label = entries[0]
                image_filepath = entries[1]
                data_dict = {
                    "image_filepath": image_filepath,
                }

                label_len = len(label)
                label_str = label[: self._max_sequence_length]
                if self._limit_max_sequence_length:
                    label_str = label_str[: self._max_sequence_length]
                    new_label_len = len(label_str)
                    if label_len != new_label_len:
                        warnings.warn(
                            f"Had to truncate image label {row['word_id']}, label string too long "
                            f"(was {label_len}, limit {self._max_sequence_length})"
                        )

                elif label_len > self._max_sequence_length:
                    self._max_sequence_length = label_len

                chars.update(label_str)

                data_dict["label"] = label_str
                self._data.append(data_dict)

        # Could be an issue if the train set doesn't have some symbols.
        # NOTE index 0 is our blank space index
        if " " in chars:
            chars.remove(" ")

        self.vocab = Vocabulary(["<SEP>"] + sorted(chars))

    # TODO num_classes and max_seq_len should be precalculated
    @property
    def num_classes(self):
        return self.vocab.num_classes

    @property
    def max_sequence_len(self):
        return self._max_sequence_length

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        data_at_idx = self._data[idx]
        image_filepath = data_at_idx["image_filepath"]
        image_label = data_at_idx["label"]

        image = cv2.imread(image_filepath, cv2.IMREAD_GRAYSCALE)
        image = resize_image(image)

        if image is None:
          print(f'Trouble reading {image_filepath}')

        # invert the image
        image = image.max() - image
        if self._transform:
            image = self._transform(image=image)['image']
            image = image.permute(1, 0)

        label_indices, label_length = self.vocab.encode(image_label)

        return {
            "image": image,
            "label_indices": label_indices,
            "label_length": label_length,
            "label_str": image_label,
        }

In [ ]:
def statX(x):
  print(f"max={torch.max(x)} min={torch.min(x)}, mean={torch.mean(x):.2f}, var={torch.var(x):.2f}, std={torch.std(x):.2f}")

In [ ]:
train_transform = A.Compose([
    #A.ShiftScaleRotate(shift_limit=0, scale_limit=0, rotate_limit=3, p=0.5),
    #A.OpticalDistortion(p=0.5),
    A.Normalize(mean=[0.5], std=[0.5]),
    ToTensor(),
])

In [ ]:
MAX_SEQ_LEN = 53
trainset = IAMDataset(
    "train.txt",
    transform=train_transform,
    max_sequence_len=MAX_SEQ_LEN,
)

In [ ]:
train_iter = iter(trainset)

In [ ]:
for _ in range(10):
  data = next(train_iter)
  print(data['label_str'])
  plt.imshow(data['image'], cmap='gray')
  plt.show()
  statX(data['image'])
  

In [ ]:
len(trainset.vocab.alphabet)

In [ ]:
trainset.vocab.alphabet.index("<SEP>")

In [ ]:
val_transform = A.Compose([
    A.Normalize(mean=[0.5], std=[0.5]),
    ToTensor(),
])

In [ ]:
testset = IAMDataset(
    "test.txt",
    transform=val_transform,
    max_sequence_len=MAX_SEQ_LEN,
)

In [ ]:
if do_data_analysis:
  max_h=0
  max_w=0
  max_l=0
  vocab = set()
  count=0
  for data in trainset:
    img = data['image']
    max_w = max(max_w, img.shape[1])
    max_h = max(max_h, img.shape[0])
    text = data['label_str']
    max_l = max(max_l, len(text))
    for c in text:
      if c not in vocab:
        vocab.add(c)
  print(f'Max height={max_h}, width={max_w}, label={max_l}')
  vocab = list(vocab)
  vocab.sort()
  print(''.join(vocab))

In [ ]:
if do_data_analysis:
  max_h=0
  max_w=0
  max_l=0
  vocab = set()
  count=0
  for data in testset:
    img = data['image']
    max_w = max(max_w, img.shape[1])
    max_h = max(max_h, img.shape[0])
    text = data['label_str']
    max_l = max(max_l, len(text))
    for c in text:
      if c not in vocab:
        vocab.add(c)
  print(f'Max height={max_h}, width={max_w}, label={max_l}')
  vocab = list(vocab)
  vocab.sort()
  print(''.join(vocab))

In [ ]:
def count_parameters(model):
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return params / 1000000

In [ ]:
num_classes = len(trainset.vocab.alphabet)
blank_label = trainset.vocab.alphabet.index("<SEP>")
gru_input_size = 256
gru_hidden_size = 256
gru_num_layers = 4

class CRNN(nn.Module):

    def __init__(self):
        super(CRNN, self).__init__()
        self.conv = EfficientNet.from_pretrained('efficientnet-b0', in_channels=1)
        self.gru = nn.GRU(gru_input_size, gru_hidden_size, gru_num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(gru_hidden_size * 2, num_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        out = self.conv.extract_features(x)
        #print(f"After CNN: {out.shape}")
        out = out.reshape(batch_size, -1, gru_input_size)
        #print(f"After reshape: {out.shape}")
        out, _ = self.gru(out)
        #print(f"After GRU: {out.shape}")
        out = torch.stack([F.log_softmax(self.fc(out[i]), dim=-1) for i in range(out.shape[0])])
        return out.permute(1, 0 , 2)

In [ ]:
model = CRNN()

In [ ]:
x = torch.randn(1,1,*IMG_SIZE)
out = model(x)
out.shape

In [ ]:
model = model.cuda()

In [ ]:
# load pre-trained weights
if load_weights and os.path.exists(weigths_file):
  model.load_state_dict(torch.load(weigths_file))

In [ ]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, collate_fn=base_collator_with_padding("label_indices"), shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, collate_fn=base_collator_with_padding("label_indices"), shuffle=False)

In [ ]:
criterion = nn.CTCLoss(blank=blank_label, reduction='mean', zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=3, verbose=True)

In [ ]:
print(f"Model has {count_parameters(model):.2f}M parameters")

In [ ]:
def create_logit_lengths(batch_size, sequence_length):
    return torch.full((batch_size,), sequence_length, dtype=torch.int64)

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
epochs=200
focal_loss = False
# ================================================ TRAINING MODEL ======================================================
early_stopping =  EarlyStopping(verbose=True, path=weigths_file)

for epoch in range(epochs):
    # ============================================ TRAINING ============================================================
    train_correct = 0
    train_total = 0
    model.train()
    train_loss = 0.0
    train_ed = 0
    train_cc = 0
    for data in tqdm(train_loader,
                     position=0, leave=True,
                     file=sys.stdout, bar_format="{l_bar}%s{bar}%s{r_bar}" % (Fore.GREEN, Fore.RESET)):
        tensor_keys = ["image", "label_indices", "label_length"]
        images, label_indices, label_lengths = tuple(
            data[k].cuda() for k in tensor_keys
        )

        b, h, w = images.shape
        images = images.reshape(b, 1 , h, w)

        optimizer.zero_grad()
        
        y_pred = model(images)

        ll = create_logit_lengths(y_pred.shape[1], y_pred.shape[0]) 
        loss = criterion(y_pred, label_indices, ll, label_lengths)

        # focal loss
        if focal_loss:
          p = torch.exp(-loss)
          # alpha=0.25, gamma=0.5
          loss = 0.25 * (1 - p)**0.5 * loss
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        preds = torch.argmax(y_pred, dim=-1)
        preds_size = torch.full(
            (preds.shape[1],), fill_value=preds.shape[0], dtype=torch.int64
        )

        flattened_preds = preds.transpose(1, 0).contiguous().view(-1)
        preds_decode = trainset.vocab.decode(flattened_preds.data, preds_size.data)

        labels = data['label_str']

        for label, pred in zip(labels, preds_decode):
          train_ed += editdistance.eval(pred, label)
          train_cc += len(label)
          train_total += 1

    print(f'TRAINING. editdistance: {train_ed}/{train_total}={train_ed/train_total:.3f}')
    print(f"[{epoch}] Train loss: {train_loss:.2f}, avg. char count={train_cc/train_total:.3f}")

    # ========================= VALIDATION =====================================
    val_correct = 0
    val_total = 0
    model.eval()
    val_loss = 0.0
    val_ed = 0
    val_cc = 0
    TO_DISPLAY = 10
    learnt = set()
    best_outputs = dict()
    for k, data in enumerate(test_loader):
        tensor_keys = ["image", "label_indices", "label_length"]
        images, label_indices, label_lengths = tuple(
            data[k].cuda() for k in tensor_keys
        )

        b, h, w = images.shape
        images = images.reshape(b, 1 , h, w)
        
        with torch.set_grad_enabled(False):
            y_pred = model(images)
        
        y_pred = model(images)

        l = create_logit_lengths(y_pred.shape[1], y_pred.shape[0]) 
        loss = criterion(y_pred, label_indices, l, label_lengths)

        # focal loss
        if focal_loss:
          p = torch.exp(-loss)
          # alpha=0.25, gamma=0.5
          loss = 0.25 * (1 - p)**0.5 * loss

        val_loss += loss.item()

        preds = torch.argmax(y_pred, dim=-1)
        preds_size = torch.full(
            (preds.shape[1],), fill_value=preds.shape[0], dtype=torch.int64
        )

        flattened_preds = preds.transpose(1, 0).contiguous().view(-1)
        preds_decode = trainset.vocab.decode(flattened_preds.data, preds_size.data)

        labels = data['label_str']

        for label, pred in zip(labels, preds_decode):
          for c in pred:
            learnt.add(c)
          ed = editdistance.eval(pred, label)
          val_ed += ed
          best_outputs[k] = (ed/len(label), label, pred)
          val_cc += len(label)
          val_total += 1

    # print learnt characters
    print(f'Learnt {len(learnt)} characters: {learnt}')

    # print best scored sentences
    best_scored = [(v[1], v[2]) for k, v in sorted(best_outputs.items(), key=lambda item: item[1][0])]
    for k in range(TO_DISPLAY):
      print(f"Output={best_scored[k][1]}\t Label={best_scored[k][0]}")

    print(f'TESTING editdistance:  {val_ed}/{val_total} = {val_ed/val_total:.3f}')
    print(f"[{epoch}] Test loss: {val_loss:.2f}, avg char count: {val_cc/val_total:.3f}")

    # step scheduler using validation loss
    scheduler.step(val_loss)

    # early stopping
    early_stopping(val_loss, model)
    
    if early_stopping.early_stop:
      print(f"Early stopping, loading best weight.. , val loss={-early_stopping.best_score:.2f}")
      model.load_state_dict(torch.load(early_stopping.path))
      break